In [3]:
# idk where to put this lol
# also im just messing around here
import inspect
import dill

In [19]:
class Frame():
    # I, Noemi, refer to this as a FrameNode to remove ambiguity with FrameObjects for the time being. 
    is_global = False

    def __init__(self, name = None, bindings=None, parent=None):
        self.parent = parent
        self.children = []
        if bindings is None:
            self.bindings = {}
        self.name = name

    def bind(self, name, value):
        self.bindings[name] = value # Note: this will allow for duplicate entries. need to figure out a better way to do this. 
    
    def set_name(self, name):
        self.name = name
    
    def add_child(self, child):
        self.children.append(child)

In [26]:
# a dictionary containing frames with associated variables in the form of a tuple: (type, name, mem loc, value)
env_frames = {"global":{}}
fobj_name_pairs = {}
# associates a frame object to a frame node (or in the case of global, at least for the time being, "global" to a frame node.)
fobj_framenode_pairs = {}
env_mutables = {}
cnter = 1

def add_new_frame(name, variables, frame_dict = env_frames): # func should always refer to the caller
    # if no frame nodes have been initialized, set the frame as the global frame and the root
    if len(fobj_framenode_pairs.keys()) == 0:
        name = "global"
        fobj_framenode_pairs[name] = Frame(bindings = variables)
        # OLD: TODO: also current_fobj not listed, fix
        env_frames[name] = {"name": None, "parent": None, "parent_fobj": None, "current_fobj": None, "variables":variables}

    else: 
        f_name = "f" + name #temp
        curr_frame = inspect.currentframe().f_back
        parent_frame = curr_frame.f_back
        # TODO: why global be stupid? fix.
        parent = fobj_framenode_pairs[parent_frame] if parent_frame in fobj_framenode_pairs else fobj_framenode_pairs["global"] # frame_name_pairs[parent_frame] if parent_frame in frame_name_pairs else "global"
        
        # OLD: dictionary rep of frame associations. Use for debugging only. 
        env_frames[f_name] = {"name": name, "parent": parent, "parent_fobj": parent_frame, "current_fobj": curr_frame, "variables":variables}
        # add self to the pairing dict
        fobj_framenode_pairs[curr_frame] = Frame(name = name, bindings = variables, parent = parent)
        # modify parent to include self as a child
        parent.add_child(fobj_framenode_pairs[curr_frame])
        # OLD: dictionary rep between fobj and frame
        fobj_name_pairs[curr_frame] = name
    return fobj_framenode_pairs, env_frames, fobj_name_pairs # messy

In [27]:
import dill
import inspect

#print("global", inspect.currentframe())
def glob():
    def A():
        z = 7
        #print("AHHHH")
        def B():
            y = 4
            #print("B", dill.detect.varnames(B), inspect.currentframe().f_back)
            def C():
                x = 5
                #print("C", inspect.currentframe().f_back)
                locs = locals()
                variables = [] #(type(locs[var]), var, dill.pointers.at(locs[var]), locs[var]) for var in locs]
                fobj_framenode_pairs, env_frames, frame_name_pairs = add_new_frame("C", variables)
                return 8 + x + y + z
            variables = [] #(type(locals()[var]), var, dill.pointers.at(locals()[var]), locals()[var]) for var in locals()]
            fobj_framenode_pairs, env_frames, frame_name_pairs = add_new_frame("B", variables)
            return C
        def D():
            #print("C2", inspect.currentframe().f_back)
            locs = locals()
            variables = [(type(locs[var]), var, locs[var]) for var in locs]
            fobj_framenode_pairs, env_frames, frame_name_pairs = add_new_frame("D", variables)
            return B()
        #print("A", inspect.currentframe().f_back, inspect.currentframe())
        locs = locals()
        #variables = [(type(locs[var]), var, dill.pointers.at(locs[var]), locs[var]) for var in locs.keys()]
        variables = []
        variables = [(type(locs[var]), var, locs[var]) for var in locs.keys()] # TODO: check whether output of locs[var] is mutable
        fobj_framenode_pairs, env_frames, frame_name_pairss = add_new_frame("A", variables) # messy
        return D()()
    cat = A()
glob()
env_frames

{'global': {'name': None,
  'parent': None,
  'parent_fobj': None,
  'current_fobj': None,
  'variables': [(function,
    'D',
    <function __main__.glob.<locals>.A.<locals>.D()>),
   (function, 'B', <function __main__.glob.<locals>.A.<locals>.B()>),
   (int, 'z', 7)]},
 'fD': {'name': 'D',
  'parent': <__main__.Frame at 0x10ac89b90>,
  'parent_fobj': <frame at 0x10aca32e0, file '/var/folders/8l/7g3122r17kv266mlj9yyt2zm0000gq/T/ipykernel_36708/3049043479.py', line 34, code A>,
  'current_fobj': <frame at 0x10ab135a0, file '/var/folders/8l/7g3122r17kv266mlj9yyt2zm0000gq/T/ipykernel_36708/3049043479.py', line 27, code D>,
  'variables': [(function,
    'B',
    <function __main__.glob.<locals>.A.<locals>.B()>)]},
 'fB': {'name': 'B',
  'parent': <__main__.Frame at 0x10ac8b710>,
  'parent_fobj': <frame at 0x10ab135a0, file '/var/folders/8l/7g3122r17kv266mlj9yyt2zm0000gq/T/ipykernel_36708/3049043479.py', line 27, code D>,
  'current_fobj': <frame at 0x10ab13ae0, file '/var/folders/8l/7g31

In [6]:
env_frames

{'global': {},
 'fA': {'name': 'A',
  'parent': 'Name not found',
  'parent_fobj': <frame at 0x10a074bf0, file '/var/folders/8l/7g3122r17kv266mlj9yyt2zm0000gq/T/ipykernel_36708/1675338650.py', line 34, code glob>,
  'current_fobj': <frame at 0x10a01d620, file '/var/folders/8l/7g3122r17kv266mlj9yyt2zm0000gq/T/ipykernel_36708/1675338650.py', line 33, code A>,
  'variables': []},
 'fC2': {'name': 'C2',
  'parent': 'A',
  'parent_fobj': <frame at 0x10a01d620, file '/var/folders/8l/7g3122r17kv266mlj9yyt2zm0000gq/T/ipykernel_36708/1675338650.py', line 33, code A>,
  'current_fobj': <frame at 0x10988a190, file '/var/folders/8l/7g3122r17kv266mlj9yyt2zm0000gq/T/ipykernel_36708/1675338650.py', line 27, code C2>,
  'variables': []},
 'fB': {'name': 'B',
  'parent': 'C2',
  'parent_fobj': <frame at 0x10988a190, file '/var/folders/8l/7g3122r17kv266mlj9yyt2zm0000gq/T/ipykernel_36708/1675338650.py', line 27, code C2>,
  'current_fobj': <frame at 0x10a01dc40, file '/var/folders/8l/7g3122r17kv266mlj9yy

In [ ]:
#THROWING SHIT AT THE WALL
x = 5
def ahhh(y):
    x = y + 4
    #print(dir())
    #print(inspect.trace())
    #print(inspect.stack())
    print(inspect.currentframe().f_back)
    print(inspect.currentframe())
    return 8
#print(globals())
#print(dir())
#print(locals())
#ahhh(8)
#traceback.extract_tb()
#frame_infos = inspect.trace()
#print(inspect.trace())
#ahhh(8)
#print(len(inspect.stack()))
#print(inspect.stack())
#print(inspect.currentframe())
#aaa = inspect.getframeinfo(inspect.currentframe())
#print(type(aaa))
#print(aaa)
#print(inspect.getinnerframes(aaa))
stack = inspect.stack()
outer_frames = inspect.getouterframes(inspect.currentframe())
inner_frames = [frame for frame in stack if frame not in outer_frames]
print(inner_frames)
#print(current_frame)

In [ ]:
import inspect
import copy
import types
import dill


#Define a function we want to modify:
def test():
    print("Here")

#Run the function to check output
print("\n\nRunning Function...")
test()
#>>> Here

#Get the source code for the test function:
testSource2 = inspect.getsourcelines(test)[0]#
testSource = dill.source.getsource(test)
print("\n\ntestSource:")
print(testSource)


#Take the inner part of the source code and modify it how we want:
newtestinnersource = ''
testSourceLines = testSource.split('\n')
print("lines")
print(testSourceLines)
linenumber = 0 
for line in testSourceLines:
    if (linenumber > 0):
        if (len(line[4:]) > 0):
            newtestinnersource += line[4:] + '\n'
    linenumber += 1
#newtestinnersource += 'print("Here2")\n'
newtestinnersource += 'return 2\n'
#print("\n\nnewtestinnersource")
print("new")
print(newtestinnersource)


#Re-assign the function's code to be a compiled version of the `innersource`
code_obj = compile(newtestinnersource, '<string>', 'exec')
test.__code__ = copy.deepcopy(code_obj)
print("\n\nRunning Modified Function...")
a = test() #<- NOW HAS MODIFIED SOURCE CODE, AND PERFORMS NEW TASK
#>>>Here
#>>>Here2
testSource2
test.__code__
print("a:")
print(a)


In [112]:
x = 5
def g():
    return 5
for i in locals():
    print(i)

__name__
__doc__
__package__
__loader__
__spec__
__builtin__
__builtins__
_ih
_oh
_dh
In
Out
get_ipython
exit
quit
open
_
__
___
sys
os
__vsc_ipynb_file__
_i
_ii
_iii
_i1
A
_i2
inspect
lines
_i3
new_code_list
new_code
i
_i4
_i5
_i6
_i7
_i8
_i9
_i10
_i11
_i12
_i13
_i14
_i15
_i16
_i17
_i18
_i19
_i20
_i21
_i22
y
_i23
_23
_i24
_i25
_i26
_i27
cat
_i28
_i29
_i30
_i31
_i32
_i33
dill
_33
_i34
_34
_i35
_i36
_36
_i37
_i38
_38
_i39
_i40
_40
_i41
_i42
_42
_i43
_i44
_44
_i45
_i46
_46
_i47
_47
_i48
_48
_i49
_i50
_50
_i51
_i52
_i53
_i54
_i55
_i56
_i57
_i58
_i59
_i60
_i61
_61
_i62
_62
_i63
_i64
_i65
_i66
_i67
_i68
_i69
_i70
_i71
_i72
_i73
_i74
_i75
_i76
_i77
_i78
x
_78
_i79
_79
_i80
_i81
_i82
env_frames
env_mutables
add_new_frame
_i83
_i84
_i85
_i86
_i87
_i88
_i89
_i90
_i91
_i92
_i93
_i94
_i95
_i96
_i97
_i98
_i99
_i100
_i101
_i102
glob
_i103
_i104
_i105
_i106
_i107
_i108
_i109
_i110
g
_i111
_i112


In [71]:
def A():
    x = 5
    print("x is ", str(x))
    print(inspect.currentframe().f_locals) # !! nestedcode, !!varnames, reference, dir, locals, f_back
    print(dill.detect.varnames(A)) # returns a tuple (local vars, local vars referrenced by nested functions) 
    ref = dill.pointers.reference(x)
    print(ref)
    return x
cat = A()

x is  5
{'x': 5, 'y': [7, 2, 1, 3]}
(('x', 'y', 'ref'), ())
4629236608


In [79]:
x = [6]
x is object

False

In [81]:
x = 5
y = x
x = 8
print(y)

5


In [29]:
print("OUTPUT BEFORE:")
A()
new_code_list = inspect.getsourcelines(A)[0]
#new_code_list = lines #handle_def(save_frame, lines)
new_code = "" # new_code_list[0] 
new_code_list[1] = '    print("jadbjhwjd")\n'
#print(new_code_list)
i = 0
while i < len(new_code_list):
    new_code += new_code_list[i]
    #new_code += line + '\n'
    i += 1
#new_code += '\n'
print("new code")
###
#new_code = dill.source.getsource(A, "A")
print(new_code)
#myast = ast.parse(new_code)
#code_obj = compile(myast, 'mymodule', 'exec')
#code_obj = compile(new_code, '<string>', 'exec')
print("OUTPUT:")
exec("""
def A():
    print("hello")
y = 7
A()""")

OUTPUT BEFORE:
x is  5
new code
def A():
    print("jadbjhwjd")
    print("x is ", str(x))
    return x

OUTPUT:
hello


In [ ]:
import dill
def environment_dict(func, start_dict = {}):
    frame=inspect.currentframe().f_back
    func.__frame = frame # location in which the function was defined (make sure this is synonymous from where it is called from?? prob not)
    return func

# TODO: do this for lambda as well
def handle_def(decorator, code_list):
    """this function recursively handles define statements within functions by adding a line at the end of the 
    def that adds some lines of code to the function before it returns."""
    # does this by finding a 'def' statement, then counting the whitespaces of each following line until one has less than or equal to the amount
    # the def func had. Then inserts the new decorator line right before that.
    func_name = None
    def_whitespace_n = 0
    i = 0
    # find the first define statement.
    while not func_name or i >= len(code_list):
        if code_list[i].lstrip(' ')[:4] == "def ":
            func_name = code_list[i].lstrip(' ')[4:].split('(', 1)[0]
            def_whitespace_n = len(code_list[i]) - len(code_list[i].lstrip(' '))
        i = i + 1
    # if something was in a define statement, handle that.
    if func_name:
        # make line that will come after the define statement's end
        new_line = ' '*def_whitespace_n + func_name + '=' + decorator.__code__.co_name + '(' + func_name + ')' + '\n'
        # 
        # TODO: check to make sure this check changes as code_list gets bigger
        #
        while i < len(code_list):
            # check to see if the amount of preceeding whitespace in the define statement is greater than or equal to the amount in the current line
            if def_whitespace_n >= (len(code_list[i]) - len(code_list[i].lstrip(' '))):
                code_list.insert(i, new_line)
                return code_list
            # check to see if a function was defined within the function (if so do recursion)
            elif code_list[i].lstrip(' ')[:4] == "def ":
                # replace that part of the code list with the new code list
                code_list[i - 1:] = handle_def(decorator, code_list[i - 1:]) # i - 1 so we include the def statement
            i = i + 1  
        code_list.append(new_line)  
    return code_list

In [ ]:
locs.keys()

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', 'open', '_', '__', '___', 'sys', 'os', '__vsc_ipynb_file__', '_i', '_ii', '_iii', '_i1', 'traceback', 'inspect', '_i2', 'x', 'test', '_i3', 'locs', '_i4', '_4', '_i5', '_i6', '_6', '_i7'])

# typing casting to try to get a valid Traceback

In [ ]:
import types
import typing

In [ ]:
"""FrameInfo(frame=<frame at 0x106e498a0, file '/var/folders/8l/7g3122r17kv266mlj9yyt2zm0000gq/T/ipykernel_1031/310886205.py', line 19, code <module>>, filename='/var/folders/8l/7g3122r17kv266mlj9yyt2zm0000gq/T/ipykernel_1031/310886205.py', lineno=19, function='<module>', code_context=['stack = inspect.stack()\n'], index=0, positions=Positions(lineno=19, end_lineno=19, col_offset=8, end_col_offset=23)),"""
test2 = typing.cast(types.TracebackType, inspect.getframeinfo(inspect.currentframe()))
test2.f_lineno = test2.tb_lineno
#test = types.TracebackType(test2, inspect.currentframe(), 0, 19)


AttributeError: 'Traceback' object has no attribute 'tb_lineno'

In [ ]:
type(test2)
inspect.getinnerframes(typing.cast(types.TracebackType, inspect.getframeinfo(inspect.currentframe())))

In [ ]:
traceback.format_tb(inspect.getframeinfo(inspect.currentframe()))

AttributeError: 'Traceback' object has no attribute 'tb_frame'